In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import optuna

In [2]:
Ridership_df= pd.read_csv('data/Ridership.csv')
df= Ridership_df.copy()
df.sample(5)

,Year,Month,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19
61969,2022,November,10,45,Corridor_6,y,Station_31,Midday,395,1,0
64116,2022,December,25,51,Corridor_3,n,Station_23,Weekend/Holiday,134,2,0
58391,2022,August,24,34,Corridor_3,y,Station_3,Evening,6691,14,0
41438,2021,August,19,33,Corridor_4,y,Station_3,AM Peak,48,2,1
42437,2021,September,9,36,Corridor_2,y,Station_3,AM Peak,225,12,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         64369 non-null  int64 
 1   Month        64369 non-null  object
 2   Day          64369 non-null  int64 
 3   Week Number  64369 non-null  int64 
 4   Corridor     64369 non-null  object
 5   Workday      64369 non-null  object
 6   Station      64369 non-null  object
 7   Period       64369 non-null  object
 8   Ridership    64369 non-null  int64 
 9   N_trains     64369 non-null  int64 
 10  Covid19      64369 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 5.4+ MB


In [4]:
df.duplicated().sum()

np.int64(0)

In [5]:
df.isnull().sum()

Year           0
Month          0
Day            0
Week Number    0
Corridor       0
Workday        0
Station        0
Period         0
Ridership      0
N_trains       0
Covid19        0
dtype: int64

In [3]:
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}
df['Month_Num'] = df['Month'].map(month_mapping)

# Cyclical Day conversion
def convert_day_to_circle(day):
    angle = 2 * np.pi * (day - 1) / 31
    x = np.cos(angle)
    y = np.sin(angle)
    return x, y

df['day_x'], df['day_y'] = zip(*df['Day'].map(convert_day_to_circle))

# Cyclical Week conversion
def convert_week_to_circle(week):
    angle = 2 * np.pi * (week - 1) / 53
    x = np.cos(angle)
    y = np.sin(angle)
    return x, y

df['week_x'], df['week_y'] = zip(*df['Week Number'].map(convert_week_to_circle))


# For Workday, instead of LabelEncoder then OneHotEncoder, let's just let OneHotEncoder handle it directly
categorical_features = ['Month', 'Corridor', 'Workday', 'Station', 'Period']

numerical_features_to_pass_through = [
    'Year', 'N_trains', 'Covid19', 'Ridership',
    'Month_Num', 'day_x', 'day_y', 'week_x', 'week_y']



preprocessor = ColumnTransformer(
    transformers=[
        ('', OneHotEncoder(), categorical_features),
    ],remainder='passthrough')


df = preprocessor.fit_transform(df)
df = df.toarray()
feature_names_out = list(preprocessor.get_feature_names_out())
df = pd.DataFrame(df, columns=[item.split('__')[1] for item in feature_names_out])

df['COVID_Workday'] = df['Covid19'] * df['Workday_y']

In [5]:
def feature_engineering(df):
    # Month mapping
    month_mapping = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
        'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    df['Month_Num'] = df['Month'].map(month_mapping)

    # Cyclical Day conversion
    def convert_day_to_circle(day):
        angle = 2 * np.pi * (day - 1) / 31
        x = np.cos(angle)
        y = np.sin(angle)
        return x, y

    df['day_x'], df['day_y'] = zip(*df['Day'].map(convert_day_to_circle))

    # Cyclical Week conversion
    def convert_week_to_circle(week):
        angle = 2 * np.pi * (week - 1) / 53
        x = np.cos(angle)
        y = np.sin(angle)
        return x, y

    df['week_x'], df['week_y'] = zip(*df['Week Number'].map(convert_week_to_circle))

    # Categorical features for OneHotEncoder
    categorical_features = ['Month', 'Corridor', 'Workday', 'Station', 'Period']

    # Column transformer for preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            ('', OneHotEncoder(), categorical_features),
        ], remainder='passthrough'
    )

    # Fit and transform the DataFrame
    df_transformed = preprocessor.fit_transform(df)

    # Convert the encoded features to a dense matrix
    df_transformed = df_transformed.toarray()

    # Convert to DataFrame and get feature names
    feature_names_out = list(preprocessor.get_feature_names_out())
    
    # Convert the dense matrix to a DataFrame
    df_transformed = pd.DataFrame(df_transformed, columns=[item.split('__')[1] for item in feature_names_out])

    return df_transformed

In [6]:
df= feature_engineering(df)

In [9]:
df['COVID_Workday'] = df['Covid19'] * df['Workday_y']

In [11]:
columns_to_convert = [
    'Month_April', 'Month_August', 'Month_December', 'Month_February', 
    'Month_January', 'Month_July', 'Month_June', 'Month_March', 
    'Month_May', 'Month_November', 'Month_October', 'Month_September',
    'Corridor_Corridor_1', 'Corridor_Corridor_2', 'Corridor_Corridor_3', 
    'Corridor_Corridor_4', 'Corridor_Corridor_5', 'Corridor_Corridor_6', 'Corridor_Corridor_7',
    'Station_Station_1', 'Station_Station_10', 'Station_Station_11', 'Station_Station_12', 
    'Station_Station_13', 'Station_Station_14', 'Station_Station_15', 'Station_Station_16', 
    'Station_Station_17', 'Station_Station_18', 'Station_Station_19', 'Station_Station_2', 
    'Station_Station_20', 'Station_Station_21', 'Station_Station_22', 'Station_Station_23', 
    'Station_Station_24', 'Station_Station_25', 'Station_Station_26', 'Station_Station_27', 
    'Station_Station_28', 'Station_Station_29', 'Station_Station_3', 'Station_Station_30', 
    'Station_Station_31', 'Station_Station_32', 'Station_Station_33', 'Station_Station_34', 
    'Station_Station_35', 'Station_Station_36', 'Station_Station_37', 'Station_Station_38', 
    'Station_Station_39', 'Station_Station_4', 'Station_Station_40', 'Station_Station_41', 
    'Station_Station_42', 'Station_Station_43', 'Station_Station_44', 'Station_Station_45', 
    'Station_Station_5', 'Station_Station_6', 'Station_Station_7', 'Station_Station_8', 'Station_Station_9',
    'Period_AM Peak', 'Period_Evening', 'Period_Midday', 'Period_PM Peak', 'Period_Weekend/Holiday',
    'Day',
    'Week Number',
    'Workday_n', 'Workday_y',
    'Covid19','COVID_Workday',
    'Month_Num']

columns_to_convert_02=['Year', 'Ridership', 'N_trains']


df[columns_to_convert] = df[columns_to_convert].astype('uint8')
df[columns_to_convert_02] = df[columns_to_convert_02].astype('uint16')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 83 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Month_April             64369 non-null  uint8  
 1   Month_August            64369 non-null  uint8  
 2   Month_December          64369 non-null  uint8  
 3   Month_February          64369 non-null  uint8  
 4   Month_January           64369 non-null  uint8  
 5   Month_July              64369 non-null  uint8  
 6   Month_June              64369 non-null  uint8  
 7   Month_March             64369 non-null  uint8  
 8   Month_May               64369 non-null  uint8  
 9   Month_November          64369 non-null  uint8  
 10  Month_October           64369 non-null  uint8  
 11  Month_September         64369 non-null  uint8  
 12  Corridor_Corridor_1     64369 non-null  uint8  
 13  Corridor_Corridor_2     64369 non-null  uint8  
 14  Corridor_Corridor_3     64369 non-null

## Problem One: 
Assume we want to find a general model for the number of passengers without looking at the number of passengers on previous days and only using the recorded information in the table for the same time period (except for the year and number of trains) to predict the number of required trains based on the number of passengers.

# spliting

In [13]:
X_p01 = df.drop(['Ridership', 'Year', 'N_trains'], axis=1)  
y_p01 = pd.DataFrame(df['Ridership'])

df_for_stratify_p01 = Ridership_df.copy()
stratify_key_p01 = df_for_stratify_p01['Covid19'].astype(str) + '_' + \
                   df_for_stratify_p01['Workday'] + '_' + \
                   df_for_stratify_p01['Period'] + '_' + \
                   df_for_stratify_p01['Corridor']

X_train_p01, X_test_p01, y_train_p01, y_test_p01 = train_test_split(
    X_p01, y_p01,
    test_size=0.2,
    random_state=42,
    stratify=stratify_key_p01 
)

In [14]:
train_data = lgb.Dataset(X_train_p01, label=y_train_p01)
test_data = lgb.Dataset(X_test_p01, label=y_test_p01, reference=train_data)

# Define parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'verbose': -1
}

# Train the model
model_p01 = lgb.train(params,
                 train_data,
                 num_boost_round=1000,
                 valid_sets=[test_data],
                 callbacks=[lgb.early_stopping(stopping_rounds=50)])

# Make predictions
y_pred = model_p01.predict(X_test_p01)

# Evaluate
rmse_f_p01 = np.sqrt(mean_squared_error(y_test_p01, y_pred))
r2_f_p01= r2_score(y_test_p01, y_pred)

print(f"RMSE: {rmse_f_p01:.2f}")
print(f"R²: {r2_f_p01:.4f}")

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 825.099
RMSE: 825.10
R²: 0.8129


In [15]:
def objective(trial):
    # Define parameters
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'feature_pre_filter': False  # Disable feature pre-filtering
    }

    # Convert data to LightGBM Dataset format inside the objective function
    train_data = lgb.Dataset(X_train_p01, label=y_train_p01)
    test_data = lgb.Dataset(X_test_p01, label=y_test_p01, reference=train_data)

    # Train the model
    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000,
                      valid_sets=[test_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50)])
    
    # Make predictions
    y_pred = model.predict(X_test_p01)
    
    # Return RMSE as the objective value to minimize
    return np.sqrt(mean_squared_error(y_test_p01, y_pred))

# Create a study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Best parameters
best_params = study.best_params
print("Best parameters:", best_params)


[I 2025-07-21 16:29:52,254] A new study created in memory with name: no-name-6c33bd2e-4cdb-43bc-a14e-fe483a36bde6


Training until validation scores don't improve for 50 rounds


[I 2025-07-21 16:29:54,513] Trial 0 finished with value: 1050.2862737363903 and parameters: {'num_leaves': 36, 'learning_rate': 0.04804586357958558, 'min_child_samples': 69, 'max_depth': 4, 'feature_fraction': 0.5908692365697692, 'bagging_fraction': 0.6467671516331035, 'bagging_freq': 4, 'lambda_l1': 3.1183145072715868, 'lambda_l2': 9.607963479519166}. Best is trial 0 with value: 1050.2862737363903.


Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1050.29
Training until validation scores don't improve for 50 rounds


[I 2025-07-21 16:29:56,614] Trial 1 finished with value: 986.9845532707903 and parameters: {'num_leaves': 31, 'learning_rate': 0.10340654809271028, 'min_child_samples': 45, 'max_depth': 4, 'feature_fraction': 0.8592227279999444, 'bagging_fraction': 0.762974008758165, 'bagging_freq': 6, 'lambda_l1': 2.91068730888814, 'lambda_l2': 1.4463650621852808}. Best is trial 1 with value: 986.9845532707903.


...

Early stopping, best iteration is:
[168]	valid_0's rmse: 865.152
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[421]	valid_0's rmse: 831.354


[I 2025-07-21 16:32:13,304] Trial 48 finished with value: 831.3539034195618 and parameters: {'num_leaves': 100, 'learning_rate': 0.25858111725450494, 'min_child_samples': 20, 'max_depth': 8, 'feature_fraction': 0.8746283263852219, 'bagging_fraction': 0.8088279975979993, 'bagging_freq': 0, 'lambda_l1': 7.210962490893447, 'lambda_l2': 1.3827501169804381}. Best is trial 46 with value: 822.8148046648784.


Training until validation scores don't improve for 50 rounds


[I 2025-07-21 16:32:14,872] Trial 49 finished with value: 878.7385451551069 and parameters: {'num_leaves': 97, 'learning_rate': 0.23694845609382892, 'min_child_samples': 8, 'max_depth': 6, 'feature_fraction': 0.8805695053308413, 'bagging_fraction': 0.7921449989646442, 'bagging_freq': 6, 'lambda_l1': 7.124253594403106, 'lambda_l2': 1.7109766946726916}. Best is trial 46 with value: 822.8148046648784.


Early stopping, best iteration is:
[352]	valid_0's rmse: 878.739
Best parameters: {'num_leaves': 51, 'learning_rate': 0.2763464521237804, 'min_child_samples': 27, 'max_depth': 9, 'feature_fraction': 0.9128801024106324, 'bagging_fraction': 0.8016549723459934, 'bagging_freq': 0, 'lambda_l1': 8.407846944542449, 'lambda_l2': 1.4198530959205615}


In [16]:
# Train final model with all data and best parameters
lgb_model_p01 = lgb.train(best_params,
                       lgb.Dataset(pd.concat([X_train_p01, X_test_p01]), 
                                 label=pd.concat([y_train_p01, y_test_p01])),
                       num_boost_round=1000)

y_pred_01 = lgb_model_p01.predict(X_test_p01)

# Evaluate
rmse_p01 = np.sqrt(mean_squared_error(y_test_p01, y_pred_01 ))
r2_p01 = r2_score(y_test_p01, y_pred_01 )

print(f"RMSE: {rmse_p01 :.2f}")
print(f"R²: {r2_p01 :.4f}")

RMSE: 606.44
R²: 0.8989


In [17]:
y_pred_train_p01 = pd.DataFrame(y_pred_01, columns=['Predicted_Passengers']).astype(int)

train_capacity = 600  
y_pred_train_p01['Predicted_Trains'] = np.ceil(y_pred_train_p01['Predicted_Passengers'] / train_capacity).astype(int)

# Add a Safety/Comfort Buffer (e.g., 5% more capacity)
# This allocates slightly more trains than the absolute minimum required.
safety_buffer_percentage = 0.05 
y_pred_train_p01['Buffered_Predicted_Trains'] = np.ceil(
    y_pred_train_p01['Predicted_Passengers'] * (1 + safety_buffer_percentage) / train_capacity).astype(int)

# 4. Ensure a Minimum Number of Trains (e.g., at least 1 train, even for 0 passengers)
minimum_trains_required = 1
y_pred_train_p01['Final_Predicted_Trains'] = y_pred_train_p01['Buffered_Predicted_Trains'].apply(
    lambda x: max(x, minimum_trains_required))

y_pred_train_p01.sample(10)

,Predicted_Passengers,Predicted_Trains,Buffered_Predicted_Trains,Final_Predicted_Trains
8469,334,1,1,1
6406,447,1,1,1
1580,182,1,1,1
2964,308,1,1,1
4557,2381,4,5,5
4535,7164,12,13,13
9722,444,1,1,1
8172,233,1,1,1
9912,931,2,2,2
6276,4512,8,8,8


In [18]:
df.sample(5)

,Month_April,Month_August,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,...,Week Number,Ridership,N_trains,Covid19,Month_Num,day_x,day_y,week_x,week_y,COVID_Workday
32472,0,0,1,0,0,0,0,0,0,0,...,52,44,1,1,12,-0.250653,-0.968077,0.972023,-0.234886,1
28009,0,0,0,0,0,0,0,0,0,0,...,37,1322,7,1,9,-0.050649,0.998717,-0.430065,-0.902798,1
57184,0,0,0,0,0,1,0,0,0,0,...,30,3370,9,0,7,0.688967,-0.724793,-0.956401,-0.292057,0
53697,0,0,0,0,0,0,0,0,1,0,...,19,583,7,0,5,-0.440394,0.897805,-0.533823,0.845596,0
39569,0,0,0,0,0,1,0,0,0,0,...,26,9,1,1,7,0.979530,0.201299,-0.984231,0.176890,1


# Problem Two:
 Assume we need to forecast the number of passengers for different time periods in order to allocate the appropriate number of trains one week in advance. (In this case, we are allowed to use information from previous time periods, but using the year and number of trains columns is still not permitted.)

In [36]:
def preprocess_and_split_data_p02(df, target_column='Ridership', train_size=0.8):
    # Convert Year, Month_Num, and Day to a datetime object
    df['Date'] = pd.to_datetime(
        df['Year'].astype(str) + '-' +
        df['Month_Num'].astype(str) + '-' +
        df['Day'].astype(str),
        errors='coerce'  # Handle potential invalid dates by setting them to NaT
    )

    # Drop rows with invalid dates
    df.dropna(subset=['Date'], inplace=True)

    # Sort by Date
    df.sort_values(by=['Date'], inplace=True)

    # Reset index
    df.reset_index(drop=True, inplace=True)

    # Define features (X) and target (y)
    X = df.drop(columns=[target_column, 'Year', 'N_trains', 'Date'])
    y = df[target_column]

    # Calculate split point
    split_point = int(len(df) * train_size)

    # Split the data into training and testing sets
    X_train = X.iloc[:split_point]
    y_train = y.iloc[:split_point]
    
    X_test = X.iloc[split_point:]
    y_test = y.iloc[split_point:]

    # Print shapes of the resulting datasets
    print(f"\nTrain set features (X_train) shape: {X_train.shape}")
    print(f"Test set features (X_test) shape: {X_test.shape}")
    print(f"Train set target (y_train) shape: {y_train.shape}")
    print(f"Test set target (y_test) shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [37]:
X_train_p02, X_test_p02, y_train_p02, y_test_p02 = preprocess_and_split_data_p02(df)


Train set features (X_train) shape: (51495, 80)
Test set features (X_test) shape: (12874, 80)
Train set target (y_train) shape: (51495,)
Test set target (y_test) shape: (12874,)


# Problem Three:
 Assume we need to forecast the number of passengers for different time periods in order to allocate a more accurate number of trains for the next day. (In this case, we are allowed to use information from previous time periods, but using the year and number of trains columns is still not permitted.)
